In [74]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import plotly.graph_objs as go


from collections import namedtuple
from plotly.subplots import make_subplots

rs = np.random.RandomState(seed=42)

In [38]:
def t_test_student_man(a: np.ndarray, b:np.ndarray, alternative='two-sided'):
    # Вычисление средних значений
    mean_a = np.mean(a)
    mean_b = np.mean(b)
    
    # Вычисление отклонения 
    std_a = np.std(a, ddof = 1)
    std_b = np.std(b, ddof = 1)

    # Вычисление статистики t
    st = (mean_a - mean_b) / np.sqrt(std_a ** 2 / a.shape[0] + std_b ** 2 / b.shape[0])
    
    # Вычисление числа степеней свободы
    size = a.shape[0] + b.shape[0] - 2
    
    Ttest_res = namedtuple('Ttest_indResult', 'statistic, pvalue')
    
    # Вычисление pvalue
    if alternative == 'two-sided':
        return Ttest_res(st, 2 * sps.t.cdf(st, df=size))
    if alternative == 'less':
        return Ttest_res(st, sps.t.cdf(st, df=size))
    if alternative == 'greater':
        return Ttest_res(st, 1 - sps.t.cdf(st, df=size))
    return ValueError('Incorrect type of hypotesis')

In [39]:
def run_t_test(hypothesis, alternative, sample_a, sample_b, alpha):
    print(hypothesis)
    t_st, p_value = t_test_student_man(sample_a, sample_b, alternative)
    
    print('\nРасчетные данные:')
    print(f't-статистика: {t_st}')
    print(f'p-value: {p_value}')
    
    st_st, sp_value = sps.ttest_ind(sample_a, sample_b, alternative=alternative)
    
    print('\nДанные ttest_ind')
    print(f't-статистика: {st_st}')
    print(f'p-value: {sp_value}')
    
    if np.all(np.isclose(t_st, st_st)) and np.all(np.isclose(p_value, sp_value)):
        print("\nРезультаты тестов совпадают")
    else:
        print("\nРезультаты тестов не совпадают")
        
    if p_value < alpha:
        print("\nНулевая гипотеза отклоняется")
    else:
        print("\nНулевая гипотеза не может быть отвергнута")        

In [40]:
sample_a = sps.norm.rvs(loc=1, scale=1, size=100)
sample_b = sps.norm.rvs(loc=1, scale=2, size=100)

run_t_test(
    "Двусторонняя проверка гипотезы на равенство средних значений:", 
    'two-sided', 
    sample_a, 
    sample_b,
    0.05)

Двусторонняя проверка гипотезы на равенство средних значений:

Расчетные данные:
t-статистика: -0.39582855601199496
p-value: 0.6926576953273162

Данные ttest_ind
t-статистика: -0.39582855601199496
p-value: 0.6926576953273162

Результаты тестов совпадают

Нулевая гипотеза не может быть отвергнута


In [41]:
sample_a = sps.norm.rvs(loc=1, scale=1, size=100)
sample_b = sps.norm.rvs(loc=10, scale=2, size=100)

run_t_test(
    "Проверка гипотезы mu1 < mu2:", 
    'less', 
    sample_a, 
    sample_b,
    0.05)

Проверка гипотезы mu1 < mu2:

Расчетные данные:
t-статистика: -39.35470347120897
p-value: 7.342343999696561e-96

Данные ttest_ind
t-статистика: -39.35470347120897
p-value: 7.342343999696561e-96

Результаты тестов совпадают

Нулевая гипотеза отклоняется


In [42]:
sample_a = sps.norm.rvs(loc=10, scale=1, size=100)
sample_b = sps.norm.rvs(loc=1, scale=2, size=100)

run_t_test(
    "Проверка гипотезы mu1 > mu2:", 
    'greater', 
    sample_a, 
    sample_b,
    0.05)

Проверка гипотезы mu1 > mu2:

Расчетные данные:
t-статистика: 42.699793711963096
p-value: 0.0

Данные ttest_ind
t-статистика: 42.699793711963096
p-value: 3.8652245166410014e-102

Результаты тестов совпадают

Нулевая гипотеза отклоняется


In [45]:
def get_bootstrap_sample_indices(sample_size: int, n_samples: int) -> np.ndarray:
    return np.random.randint(0, sample_size, (n_samples, sample_size))

def get_bootstrap_samples(X: np.ndarray, n_samples: int) -> np.ndarray:
    return X[get_bootstrap_sample_indices(len(X), n_samples)]

def mix_norm_dist(size: int) -> np.ndarray:
    d_1 = sps.norm.rvs(loc=1, scale=1, size=size)
    d_2 = sps.norm.rvs(loc=4, scale=0.5, size=size)
    
    a_mask = sps.randint.rvs(0, 2, size=size).astype(bool)
    return np.append(
        d_1[a_mask],
        d_2[~a_mask]
    )    

def bootstrap(X, alpha=0.05) -> None:
    X_b = get_bootstrap_samples(X, 10_000)
    
    mean_samples = np.mean(X_b, axis=1)
    median_samples = np.median(X_b, axis=1)
    
    print(f"Истинное значение среднего: {np.mean(X)}")
    print(f"Истинное значение медианы: {np.median(X)}")
    
    # Доверительный интервал для среднего
    CI_mean = np.percentile(mean_samples, (alpha * 100, (1 - alpha) * 100))
    print(f"\nДоверительный интервал для среднего {CI_mean}")

    # Доверительный интервал для медианы
    CI_median = np.percentile(median_samples, (alpha * 100, (1 - alpha) * 100))
    print(f"Доверительный интервал для медианы {CI_median}")
    
    percent_check_mean = np.sum(
            [CI_mean[0] <= mean <= CI_mean[1] for mean in mean_samples]
        ) / mean_samples.shape[0]
    percent_check_med = np.sum(
        [CI_median[0] <= median <= CI_median[1] for median in median_samples]
        ) / median_samples.shape[0]
    print(f"\nПроцент средних попавших в интервал: {percent_check_mean}")
    print(f"Процент медиан попавших в интервал: {percent_check_med}")
    

In [46]:
size = 1000
x_a = sps.expon.rvs(loc=4, scale=16, size=size)
bootstrap(x_a, 0.05)

Истинное значение среднего: 19.329559596507185
Истинное значение медианы: 14.834552982213498

Доверительный интервал для среднего [18.53359427 20.13463997]
Доверительный интервал для медианы [13.8847537  15.95144658]

Процент средних попавших в интервал: 0.9
Процент медиан попавших в интервал: 0.9015


In [47]:
size = 1000
x_a = sps.norm.rvs(loc=1, scale=2, size=size)
bootstrap(x_a, 0.05)

Истинное значение среднего: 1.030883130823787
Истинное значение медианы: 1.0275095589578365

Доверительный интервал для среднего [0.92661402 1.1364124 ]
Доверительный интервал для медианы [0.91016372 1.16922818]

Процент средних попавших в интервал: 0.9
Процент медиан попавших в интервал: 0.9038


In [48]:
x_a = mix_norm_dist(500)
bootstrap(x_a, 0.05)

Истинное значение среднего: 2.520411588069363
Истинное значение медианы: 2.932431895453261

Доверительный интервал для среднего [2.39632125 2.64704071]
Доверительный интервал для медианы [2.46509598 3.31883353]

Процент средних попавших в интервал: 0.9
Процент медиан попавших в интервал: 0.9033


In [49]:
def criterion_power_t(x_1, x_2, n_samples, n_exp, alpha=0.05, equal_var=True):
   
    p_values = []
    for _ in range(n_exp):
        sample_1 = x_1.rvs(size=n_samples)
        sample_2 = x_2.rvs(size=n_samples)
        
        p_values.append(sps.ttest_ind(sample_1, sample_2, equal_var=equal_var).pvalue)
    p_values = np.array(p_values)
    
    return p_values[p_values < alpha].shape[0] / p_values.shape[0]

def criterion_power_mv(x_1, x_2, n_samples, n_exp, alpha=0.05):
    
    p_values = []
    for _ in range(n_exp):
        sample_1 = x_1.rvs(size=n_samples)
        sample_2 = x_2.rvs(size=n_samples)
        
        p_values.append(sps.mannwhitneyu(sample_1, sample_2).pvalue)
    p_values = np.array(p_values)
    
    return p_values[p_values < alpha].shape[0] / p_values.shape[0]    

In [65]:
size = 100

effects = [0, 0.02, 0.4, 0.7, 1, 1.4]


ttest_power = []
mv_power = []

for effect in effects:
    x_a = sps.norm(loc=1, scale=1)
    x_b = sps.norm(loc=1 + effect, scale=1)
    
    ttest_power.append(criterion_power_t(x_a, x_b, size, 1000, 0.05, False) * 100)
    mv_power.append(criterion_power_mv(x_a, x_b, size, 1000, 0.05) * 100)
print(f"Effects: {effects}")
print(f"Ttest: {np.round(np.array(ttest_power), 3)}")
print(f"Mann-Whitheyu: {np.round(np.array(mv_power), 3)}")

Effects: [0, 0.02, 0.4, 0.7, 1, 1.4]
Ttest: [  4.2   5.7  79.6  99.8 100.  100. ]
Mann-Whitheyu: [  3.8   4.9  78.3  99.8 100.  100. ]


In [64]:
size = 100

effects = [0, 0.02, 0.4, 0.7, 1, 1.4]

ttest_power = []
mv_power = []

for effect in effects:
    x_a = sps.norm(loc=1, scale=1)
    x_b = sps.norm(loc=1 + effect, scale=14)
    
    ttest_power.append(criterion_power_t(x_a, x_b, size, 1000, 0.05, False) * 100)
    mv_power.append(criterion_power_mv(x_a, x_b, size, 1000, 0.05) * 100)
print(f"Effects: {effects}")
print(f"Ttest: {np.round(np.array(ttest_power), 3)}")
print(f"Mann-Whitheyu: {np.round(np.array(mv_power), 3)}")

Effects: [0, 0.02, 0.4, 0.7, 1, 1.4]
Ttest: [ 5.1  4.9  5.9  7.4 11.6 16.7]
Mann-Whitheyu: [ 9.2 10.4  8.  12.  13.  23. ]


In [66]:
size = 100

effects = [0, 0.02, 0.4, 0.7, 1, 1.4]

ttest_power = []
mv_power = []

for effect in effects:
    x_a = sps.expon(loc=1, scale=1)
    x_b = sps.expon(loc=1. + effect, scale=1)
    
    ttest_power.append(criterion_power_t(x_a, x_b, size, 1000, 0.05, False) * 100)
    mv_power.append(criterion_power_mv(x_a, x_b, size, 1000, 0.05) * 100)
print(f"Effects: {effects}")
print(f"Ttest: {np.round(np.array(ttest_power), 3)}")
print(f"Mann-Whitheyu: {np.round(np.array(mv_power), 3)}")

Effects: [0, 0.02, 0.4, 0.7, 1, 1.4]
Ttest: [  6.3   4.6  79.4  99.8 100.  100. ]
Mann-Whitheyu: [  5.3   7.2  98.6 100.  100.  100. ]


In [68]:
size = 100

effects = [0, 0.02, 0.04, 0.07, 0.1, 0.14]

ttest_power = []
mv_power = []

for effect in effects:
    x_a = sps.bernoulli(p=0.015)
    x_b = sps.bernoulli(p=0.015 + effect)
    
    ttest_power.append(criterion_power_t(x_a, x_b, size, 1000, 0.05, False) * 100)
    mv_power.append(criterion_power_mv(x_a, x_b, size, 1000, 0.05) * 100)
print(f"Effects: {effects}")
print(f"Ttest: {np.round(np.array(ttest_power), 3)}")
print(f"Mann-Whitheyu: {np.round(np.array(mv_power), 3)}")

Effects: [0, 0.02, 0.04, 0.07, 0.1, 0.14]
Ttest: [ 1.8 13.8 32.6 62.7 85.9 97.9]
Mann-Whitheyu: [ 3.  12.6 28.5 63.8 86.4 97.4]


In [93]:
def qq_plot(x_1, x_2):
    p_values = []
    p_values_mw = []
    size = 100
    n_exp = 1_000
    for _ in range(n_exp):
        x_a = x_1.rvs(size=size)
        x_b = x_2.rvs(size=size)
        p_values.append(sps.ttest_ind(x_a, x_b, equal_var=False).pvalue)
        p_values_mw.append(sps.mannwhitneyu(x_a, x_b).pvalue)
    p_values = np.array(p_values)
    p_values_mw = np.array(p_values_mw)
    
    probs = []
    probs_mw = []
    x = [0.01 * i for i in range(101)]
    
    for i in range(101):
        alpha_step = 0.01 * i
        probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])
        probs_mw.append(p_values_mw[p_values_mw < alpha_step].shape[0] / p_values_mw.shape[0])
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=("P-value hist Ttest", "P-value hist Mann-Whitneyu"))
        
    fig.add_trace(
        go.Histogram(
            x=probs,
            xbins={"start":0, "end":1, "size":0.1},
            histnorm='probability'),
        row=1, col=1)   
    
    fig.add_trace(
        go.Histogram(
            x=probs_mw,
            xbins={"start":0, "end":1, "size":0.1},
            histnorm='probability'),
        row=1, col=2)
                      
    fig.update_layout(height=400, width=800)
    fig.show()


    fig = make_subplots(rows=1, cols=2, subplot_titles=("Q-Q plot Ttest", "Q-Q plot Mann-Whitneyu"))
        
    fig.add_trace(go.Scatter(x=x, y=probs, name='p_value', mode="markers"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=x, mode="lines", name="uniform"), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=x, y=probs_mw, name='p_value', mode="markers"), row=1, col=2)
    fig.add_trace(go.Scatter(x=x, y=x, mode="lines", name="uniform"), row=1, col=2)
                      
    fig.update_layout(height=400, width=800)
    fig.show()

In [94]:
size = 100

x_a = sps.norm(loc=1, scale=1)
x_b = sps.norm(loc=1, scale=1)

qq_plot(x_a, x_b)


In [95]:
size = 100

x_a = sps.expon(scale=1/2)
x_b = sps.expon(scale=1/2)

qq_plot(x_a, x_b)